In [3]:
import os, sys
sys.path.append('C:/Users/sumaiyah/OneDrive - University Of Cambridge/Project/NN-Builder/src/preprocessing/MB-PAM50')

In [4]:
import pandas as pd
import numpy as np

In [5]:
from helpers import get_data

# Make Pam50 binary

## Get data

In [6]:
# Returns train or test data from a specific fold
get_data_from_fold = lambda data_type, fold_index: pd.read_csv('5-fold_pam50stratified/fold%s/MBdata_33CLINwMiss_1KfGE_1KfCNA_%s.csv' % (str(fold_index), data_type))

In [7]:
# target_col = 'ernp'
target_col = 'pam50np'

In [8]:
train_data = get_data(get_data_from_fold('train', 1))
test_data = get_data(get_data_from_fold('test', 1))

c:\users\sumaiyah\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
def remove_any_rows_with_unknowns(df):
    for col in df.columns:
        df.drop(df[df[col]=='?'].index, inplace=True)
    return df

In [10]:
# Concatenate gene expression data and clinical data for input to pam50
# train_data_df = pd.DataFrame(train_data['rnanp'])
train_data_df = pd.concat([pd.DataFrame(train_data['rnanp']),pd.DataFrame(train_data['clin'])], axis=1, ignore_index=True)
train_data_df['target'] = train_data[target_col]
train_data_df = remove_any_rows_with_unknowns(train_data_df)
train_data_df.to_csv('train_data.csv', index=False)

# test_data_df = pd.DataFrame(test_data['rnanp'])
test_data_df = pd.concat([pd.DataFrame(test_data['rnanp']),pd.DataFrame(test_data['clin'])], axis=1, ignore_index=True)
test_data_df['target'] = test_data[target_col]
test_data_df = remove_any_rows_with_unknowns(test_data_df)
test_data_df.to_csv('test_data.csv', index=False)

print(train_data_df.shape)
print(test_data_df.shape)

c:\users\sumaiyah\appdata\local\programs\python\python36\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


(1579, 1351)
(395, 1351)


In [18]:
train_data_df[train_data_df['target'] == 1]

0       False
1       False
2       False
3       False
4       False
        ...  
1574    False
1575    False
1576    False
1577    False
1578    False
Name: target, Length: 1579, dtype: bool

## Build Network

In [66]:
from typing import List
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense

def build_network(structure: List[int], activation_function: str, loss='categorical_crossentropy', optimizer='adam'):
    # Assuming structure is none empty TODO handle case none empty structure
    """
    Args:
        structure: structure of network INCLUDING input and output layer
        activation_function:
        loss:
        optimizer:

    Returns:
        Compiled Keras model based on specification
    """

    # Input layer
    input_layer = Input(shape=(structure[0],))

    # Hidden Layers
    prev_layer = input_layer
    for x in range(1, len(structure) - 1):
        hidden_layer = Dense(structure[x], activation=activation_function)(prev_layer)
        prev_layer = hidden_layer

    # Output Layer
    output_layer = Dense(structure[len(structure) - 1], activation='softmax')(prev_layer)

    # Model input-hidden-output
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile Model
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=['accuracy'])

    return model

In [67]:
X_train = train_data_df.drop(['target'], axis=1).iloc[:, :].values
y_train = train_data_df['target'].values

X_test = test_data_df.drop(['target'], axis=1).iloc[:, :].values
y_test = test_data_df['target'].values

In [68]:
from keras.utils import to_categorical
from sklearn.utils import class_weight

# Weight classes due to imbalanced dataset
class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)))

In [71]:
model = build_network(structure=[1350, 1000, 300, 100, 5], activation_function='relu')
batch_size = 25
epochs = 200

In [ ]:
# Train Model
model.fit(X_train,
          to_categorical(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          class_weight=class_weights)

# Test Model
_, accuracy = model.evaluate(X_test, to_categorical(y_test))

print(accuracy)

In [48]:
# Save Model
model.save('model.h5')

In [49]:
# Save Network Predictions
network_predictions = np.argmax(model.predict(X_test), axis=1)
with open(('NN_predictions.txt'), 'w') as file:
    file.write(' '.join([str(pred) for pred in network_predictions]))